# Colab 前置處理(一定要先執行以下所有程式)

- 取得雲端存取權限
  - 會出現網址要求登入(2次)，此時要把驗證碼複製並貼回。
- 建立相關暫存資料夾
- 將目錄移至FDA_final
- 下載套件

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144467 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.22-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.22-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p Drive
!google-drive-ocamlfuse Drive

In [0]:
import os
#將資料目錄移至FDA_final
os.chdir("Drive/Colab Notebooks/FDA_final")

In [4]:
pip install -r requirements.txt

     |████████████████████████████████| 20.2MB 1.4MB/s 
     |████████████████████████████████| 10.0MB 42.5MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.4
    Uninstalling numpy-1.18.4:
      Successfully uninstalled numpy-1.18.4
  Found existing installation: pandas 1.0.4
    Uninstalling pandas-1.0.4:
      Successfully uninstalled pandas-1.0.4


# 觀察與分析
## 天氣與確診數之間的關係
* 資料來源: https://www.kaggle.com/kimjihoo/coronavirusdataset/data
* 日期: 2020-01-20 到 2020-05-13
*  16 個地方
    1. Seoul
    2. Busan
    3. Daegu 
    4. Gwangju
    5. Incheon
    6. Daejeon
    7. Ulsan 	
    8. Gyeonggi-do 
    9. Gangwon-do
    11. Chungcheongnam-do 
    12. Jeollabuk-do
    13. Jeollanam-do
    14. Gyeongsangbuk-do
    15. Gyeongsangnam-do
    16. Jeju-do 	
* 溼度與卻診數的關係
* 溫度與卻診數的關係

In [0]:
# 匯入套件
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [6]:
# 讀取 Weather
df = pd.read_csv('Weather.csv')
df['date'] = pd.to_datetime(df['date'])
df  = df[df.date.dt.year == 2020]

# 只抽取 2020 年的
df.head(5)

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
23375,10000,Seoul,2020-01-01,-2.2,-6.5,0.3,0.1,2.6,50.0,64.4
23376,11000,Busan,2020-01-01,1.9,-3.2,7.8,0.0,5.1,340.0,44.0
23377,12000,Daegu,2020-01-01,0.2,-4.9,4.6,0.0,5.6,270.0,53.3
23378,13000,Gwangju,2020-01-01,-0.3,-4.9,5.7,0.0,4.3,50.0,58.0
23379,14000,Incheon,2020-01-01,-1.4,-5.4,1.9,0.0,3.8,160.0,66.6


In [7]:
# 讀取 province
province = pd.read_csv('TimeProvince.csv')
province['date'] = pd.to_datetime(province['date'])
province.head(5)

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0


In [8]:
province.tail(5)
# 可得知要取出的日期介於 2020-01-20 到 2020-05-13

,date,time,province,confirmed,released,deceased
1967,2020-05-14,0,Jeollabuk-do,20,14,0
1968,2020-05-14,0,Jeollanam-do,17,13,0
1969,2020-05-14,0,Gyeongsangbuk-do,1368,1237,54
1970,2020-05-14,0,Gyeongsangnam-do,119,114,0
1971,2020-05-14,0,Jeju-do,14,12,0


In [0]:
# 取出的日期介於 2020-01-20 到 2020-05-13
df = df[df['date'] >= '2020-01-20']
df = df[df['date'] <= '2020-05-13']

In [10]:
df.head(5)

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
23679,10000,Seoul,2020-01-20,0.7,-3.3,5.5,0.0,5.7,270.0,46.8
23680,11000,Busan,2020-01-20,5.1,1.6,9.8,0.0,6.1,270.0,45.6
23681,12000,Daegu,2020-01-20,4.6,2.4,8.1,0.0,6.4,290.0,44.1
23682,13000,Gwangju,2020-01-20,4.0,1.2,7.9,0.0,3.3,270.0,63.1
23683,14000,Incheon,2020-01-20,1.2,-2.1,5.1,0.0,8.6,320.0,56.0


In [11]:
df.tail()

,code,province,date,avg_temp,min_temp,max_temp,precipitation,max_wind_speed,most_wind_direction,avg_relative_humidity
25514,50000,Jeollabuk-do,2020-05-13,16.1,9.2,22.1,0.0,4.7,110.0,50.9
25515,51000,Jeollanam-do,2020-05-13,18.0,12.7,23.4,0.0,9.0,230.0,46.5
25516,60000,Gyeongsangbuk-do,2020-05-13,18.8,12.3,24.3,0.0,8.0,290.0,37.4
25517,61000,Gyeongsangnam-do,2020-05-13,17.4,10.7,24.0,0.0,4.3,160.0,46.3
25518,70000,Jeju-do,2020-05-13,17.0,13.2,20.4,0.0,4.1,320.0,47.4


In [0]:
province = province[province['date'] >= '2020-01-20']
province = province[province['date'] <= '2020-05-13']

In [13]:
province.head(5)

,date,time,province,confirmed,released,deceased
0,2020-01-20,16,Seoul,0,0,0
1,2020-01-20,16,Busan,0,0,0
2,2020-01-20,16,Daegu,0,0,0
3,2020-01-20,16,Incheon,1,0,0
4,2020-01-20,16,Gwangju,0,0,0


In [14]:
province.tail(5)

,date,time,province,confirmed,released,deceased
1950,2020-05-13,0,Jeollabuk-do,20,14,0
1951,2020-05-13,0,Jeollanam-do,16,12,0
1952,2020-05-13,0,Gyeongsangbuk-do,1367,1231,54
1953,2020-05-13,0,Gyeongsangnam-do,119,113,0
1954,2020-05-13,0,Jeju-do,14,12,0


In [15]:
# 各個地方的 dataframe
weather = []
sick = []
location = ['Seoul', 'Busan', 'Daegu', 'Gwangju', 'Incheon', 'Daejeon', 'Ulsan', 'Gyeonggi-do', 'Gangwon-do', 'Chungcheongnam-do', 'Jeollabuk-do', 'Jeollanam-do', 'Gyeongsangbuk-do', 'Gyeongsangnam-do', 'Jeju-do']
for i in range(0,len(location)):
    weather.append(df[df.province == location[i]])
    sick.append(province[province.province == location[i]])
print('---------\nWeather.csv\n----------')
for i in range(0,len(weather)):
    print('Province:{} 有{}筆資料'.format(weather[i].iloc[0,1], weather[i].shape[0]))
print('---------\nTimeProvince.csv\n----------')
for i in range(0,len(weather)):
    print('Province:{} 有{}筆資料'.format(sick[i].iloc[0,2], sick[i].shape[0]))

---------
Weather.csv
----------
Province:Seoul 有115筆資料
Province:Busan 有115筆資料
Province:Daegu 有115筆資料
Province:Gwangju 有115筆資料
Province:Incheon 有115筆資料
Province:Daejeon 有115筆資料
Province:Ulsan 有115筆資料
Province:Gyeonggi-do 有115筆資料
Province:Gangwon-do 有115筆資料
Province:Chungcheongnam-do 有115筆資料
Province:Jeollabuk-do 有115筆資料
Province:Jeollanam-do 有115筆資料
Province:Gyeongsangbuk-do 有115筆資料
Province:Gyeongsangnam-do 有115筆資料
Province:Jeju-do 有115筆資料
---------
TimeProvince.csv
----------
Province:Seoul 有115筆資料
Province:Busan 有115筆資料
Province:Daegu 有115筆資料
Province:Gwangju 有115筆資料
Province:Incheon 有115筆資料
Province:Daejeon 有115筆資料
Province:Ulsan 有115筆資料
Province:Gyeonggi-do 有115筆資料
Province:Gangwon-do 有115筆資料
Province:Chungcheongnam-do 有115筆資料
Province:Jeollabuk-do 有115筆資料
Province:Jeollanam-do 有115筆資料
Province:Gyeongsangbuk-do 有115筆資料
Province:Gyeongsangnam-do 有115筆資料
Province:Jeju-do 有115筆資料


# 溫度與確診數的關係
* 15 個地區 & 全國
* 累計確診數 & 當日確診數

In [16]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline as offline

subplot_titles_m = []
specs_m = []

for i in range(0,len(location)):
  subplot_titles_m.append(location[i])
  specs_m.append([{"secondary_y": True}])

fig = make_subplots(rows=len(location), cols=1, subplot_titles=subplot_titles_m, specs=specs_m)

# 累計確診數
for i in range(0,len(location)):
  show_legend_f = False
  if i == 0:
    show_legend_f = True
  fig.add_trace(
      go.Scatter(name = "Comfirmed(culmulative)", x=sick[i].date, y = sick[i].confirmed, showlegend=show_legend_f, line=dict(color='red', width=1)),
      row=i+1, col=1,
      secondary_y=False
      )
  fig.add_trace(
      go.Scatter(name = "Average temperature",x = weather[i].date, y = weather[i].avg_temp, showlegend=show_legend_f, line=dict(color='blue', width=1)),
      row=i+1, col=1,
      secondary_y=True
      )
fig.update_layout(height=4000, width=800,title_text="Temperature versus Comfirmed")
fig.show()
offline.plot(fig)


'temp-plot.html'

![E.png](graph/E.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/E.html

In [17]:
# 當日確診數
subplot_titles_m = []
specs_m = []

for i in range(0,len(location)):
  subplot_titles_m.append(location[i])
  specs_m.append([{"secondary_y": True}])

fig = make_subplots(rows=len(location), cols=1, subplot_titles=subplot_titles_m, specs=specs_m)

for i in range(0,len(location)):
  temp=sick[i].confirmed.shift(1)
  show_legend_f = False
  if i == 0:
    show_legend_f = True
  fig.add_trace(
      go.Scatter(name = "Comfirmed(Single Day)", x=sick[i].date, y = sick[i].confirmed-temp, showlegend=show_legend_f, line=dict(color='red', width=1)),
      row=i+1, col=1,
      secondary_y=False
      )
  fig.add_trace(
      go.Scatter(name = "Average temperature",x = weather[i].date, y = weather[i].avg_temp, showlegend=show_legend_f, line=dict(color='blue', width=1)),
      row=i+1, col=1,
      secondary_y=True
      )
fig.update_layout(height=4000, width=800,title_text="Temperature versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![F.png](graph/F.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/F.html

# 溼度與確診數的關係
* 15 個地區 & 全國
* 累計確診數 & 當日確診數

In [18]:
# 累積確診數
subplot_titles_m = []
specs_m = []

for i in range(0,len(location)):
  subplot_titles_m.append(location[i])
  specs_m.append([{"secondary_y": True}])

fig = make_subplots(rows=len(location), cols=1, subplot_titles=subplot_titles_m, specs=specs_m)

for i in range(0,len(location)):
  show_legend_f = False
  if i == 0:
    show_legend_f = True
  fig.add_trace(
      go.Scatter(name = "Comfirmed(culmulative)", x=sick[i].date, y = sick[i].confirmed, showlegend=show_legend_f, line=dict(color='red', width=1)),
      row=i+1, col=1,
      secondary_y=False
      )
  fig.add_trace(
      go.Scatter(name = "Average relative humidity",x = weather[i].date, y = weather[i].avg_relative_humidity, showlegend=show_legend_f, line=dict(color='blue', width=1)),
      row=i+1, col=1,
      secondary_y=True
      )
fig.update_layout(height=4000, width=800,title_text="Humidity versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![G.png](graph/G.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/G.html

In [19]:
# 當日確診數
subplot_titles_m = []
specs_m = []

for i in range(0,len(location)):
  subplot_titles_m.append(location[i])
  specs_m.append([{"secondary_y": True}])

fig = make_subplots(rows=len(location), cols=1, subplot_titles=subplot_titles_m, specs=specs_m)

for i in range(0,len(location)):
  temp=sick[i].confirmed.shift(1)
  show_legend_f = False
  if i == 0:
    show_legend_f = True
  fig.add_trace(
      go.Scatter(name = "Comfirmed(Single Day)", x=sick[i].date, y = sick[i].confirmed-temp, showlegend=show_legend_f, line=dict(color='red', width=1)),
      row=i+1, col=1,
      secondary_y=False
      )
  fig.add_trace(
      go.Scatter(name = "Average relative humidity",x = weather[i].date, y = weather[i].avg_relative_humidity, showlegend=show_legend_f, line=dict(color='blue', width=1)),
      row=i+1, col=1,
      secondary_y=True
      )
fig.update_layout(height=4000, width=800,title_text="Humidity versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![H.png](graph/H.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/H.html

# 全國

In [0]:
# 全國
for i in range(0,len(location)):
    weather[i].set_index('date',inplace=True)
    sick[i].set_index('date',inplace=True)

In [21]:
korea = pd.DataFrame(index=sick[0].index,columns=['confirmed','avg_temp','avg_relative_humidity'])
korea.confirmed = sick[0].confirmed
korea.avg_temp = weather[0].avg_temp
korea.avg_relative_humidity = weather[0].avg_relative_humidity
for i in range(1,len(location)):
    korea.confirmed += sick[i].confirmed
    korea.avg_temp += weather[i].avg_temp
    korea.avg_relative_humidity += weather[i].avg_relative_humidity
korea.avg_temp /= len(location)
korea.avg_relative_humidity /= len(location)
korea

,confirmed,avg_temp,avg_relative_humidity
date,,,
2020-01-20,1,3.646667,52.606667
2020-01-21,1,2.173333,47.566667
2020-01-22,1,4.733333,67.906667
2020-01-23,1,6.506667,78.020000
2020-01-24,2,5.913333,70.320000
...,...,...,...
2020-05-09,10277,16.146667,93.160000
2020-05-10,10303,16.286667,81.580000
2020-05-11,10331,18.573333,60.813333


In [22]:
# 溫度:累計
fig = make_subplots(subplot_titles=["Korea (country)"], specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(name = "Comfirmed(culmulative)", x = korea.index, y = korea['confirmed'], line=dict(color='red', width=1)),
    secondary_y=False
    )
fig.add_trace(
    go.Scatter(name = "Average temperature", x = korea.index, y = korea['avg_temp'], line=dict(color='blue', width=1)),
    secondary_y=True
    )
fig.update_layout(height=800, width=1000,title_text="Temperature versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![I.png](graph/I.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/I.html

In [23]:
# 溫度:每日
temp=korea.confirmed.shift(1)
fig = make_subplots(subplot_titles=["Korea (country)"], specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(name = "Comfirmed(Single Day)", x = korea.index, y = korea.confirmed-temp, line=dict(color='red', width=1)),
    secondary_y=False
    )
fig.add_trace(
    go.Scatter(name = "Average temperature", x = korea.index, y = korea.avg_temp, line=dict(color='blue', width=1)),
    secondary_y=True
    )
fig.update_layout(height=800, width=1000,title_text="Temperature versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![J.png](graph/J.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/J.html

In [27]:
# 溼度:累計
fig = make_subplots(subplot_titles=["Korea (country)"], specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(name = "Comfirmed(culmulative)", x = korea.index, y = korea['confirmed'], line=dict(color='red', width=1)),
    secondary_y=False
    )
fig.add_trace(
    go.Scatter(name = "Average temperature", x = korea.index, y = korea['avg_relative_humidity'], line=dict(color='blue', width=1)),
    secondary_y=True
    )
fig.update_layout(height=800, width=1000,title_text="Humidity versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![K.png](graph/K.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/K.html

In [28]:
# 溼度:每日

temp=korea.confirmed.shift(1)
fig = make_subplots(subplot_titles=["Korea (country)"], specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(name = "Comfirmed(Single Day)", x = korea.index, y = korea.confirmed-temp, line=dict(color='red', width=1)),
    secondary_y=False
    )
fig.add_trace(
    go.Scatter(name = "Average temperature", x = korea.index, y = korea.avg_relative_humidity, line=dict(color='blue', width=1)),
    secondary_y=True
    )
fig.update_layout(height=800, width=1000,title_text="Humidity versus Comfirmed")
fig.show()
offline.plot(fig)

'temp-plot.html'

![L.png](graph/L.png?raw=true)

https://aaron-chang-ac.github.io/FDA_final/graph/L.html